Импортируем необходимые библиотеки

In [ ]:
import os
import pandas as pd
import numpy as np
import zipfile
from skimage import io
import matplotlib.pyplot as plt
import matplotlib
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, random_split, DataLoader
from torchmetrics.classification.accuracy import Accuracy

%matplotlib inline

Определяем константы

In [ ]:
# Количество батчей
BATCH_SIZE = 64

# Количество эпох
EPOCHS = 100

# Количество классов
OUTPUT_SIZE = 2

Определяем custom функции

In [ ]:
class WithWithoutSpecs(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = self.join_annotations(csv_file=csv_file)
        self.root_dir = root_dir
        self.transform = transform
        return

    def __len__(self):
        """
        Returns len of dataset
        """
        return len(self.annotations)

    def __getitem__(self, index):
        """
        Specific image and target for this item
        """
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 1])
        image = io.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index, 0]))
    
        if self.transform:
            image = self.transform(image)
        return (image, y_label)

    def join_annotations(self, csv_file):
        data = pd.read_csv(csv_file, usecols=['id', 'glasses'])
        data['link'] = data['id'].apply(lambda s: 'face-' + str(s) + '.png')
        data.drop(['id'], inplace=True, axis=1)
        return data


def image_shower(images, labels, n=4):
    """
    Данная функция нужна для вывода изображений
    """
    classes = ['without glasses', 'with glasses']
    fig, axs = plt.subplots(1, n, constrained_layout=True, figsize=(12, 12))
    for i, image in enumerate(images[:n]):
        image = image / 2 + 0.5
        axs[i].imshow(image.numpy().transpose((1, 2, 0)).squeeze())
        axs[i].set_title(classes[labels[i]])
        axs[i].axis('off')
        

def save_to_file(*args):
    """
    Для сохранения значений Loss и Accuracy в файлик
    """
    with open('models/v2/logs.csv', 'a') as f:
        f.write(';'.join(map(str, args)) + '\n')

        
def run_train():
    """
    Обучение модели
    """
    best_test_loss_value = float('inf')
    for epoch in range(EPOCHS): 
        
        # train часть
        running_loss = []
        running_acc = []
        
        for features, labels in tqdm(train_loader):
            # помещаем данные на GPU
            features, labels = features.to(device), labels.to(device)
            
            # сбрасываем накопленный градиент 
            optimizer.zero_grad()
            
            # предсказываем
            output = model(features)
            
            # считаем ошибку, точность и градиенты
            loss = criterion(output, labels)
            loss.backward()
            acc = accuracy(output, labels)
            
            # делаем шаг оптимизатора (обновляем веса)
            optimizer.step()

            running_loss.append(loss.item())
            running_acc.append(acc.item())
            
        train_loss_value = np.mean(running_loss)
        train_accuracy_value = np.mean(running_acc)
        
        # test часть
        running_loss = []
        running_acc = []
        
        for features, labels in test_loader:
            # помещаем данные на GPU
            features, labels = features.to(device), labels.to(device)
            
            # предсказываем
            output = model(features)
            
            # считаем ошибку и точность
            loss = criterion(output, labels)
            acc = accuracy(output, labels)

            running_loss.append(loss.item())
            running_acc.append(acc.item())
            
        test_loss_value = np.mean(running_loss)
        test_accuracy_value = np.mean(running_acc)
        
        save_to_file(epoch, train_loss_value, train_accuracy_value, test_loss_value, test_accuracy_value)
        if test_loss_value < best_test_loss_value:
            model_path = os.path.join('models\\v2', 'model.pth')
            torch.save(model, model_path)
            best_test_loss_value = test_loss_value
                
    return

Установить GPU как device для вычислений PyTorch

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
device

Создадим свой датасет с картинками

Разобьем его на тренировочную и валидационную выборку

Создадим DataLoader для тренировочной и тестовой выборок

**Датасет для обучения взят <a href='https://www.kaggle.com/jeffheaton/glasses-or-no-glasses'>отсюда</a>**

In [ ]:
transform = transforms.Compose(
    [transforms.ToPILImage(), 
     transforms.Resize((64, 64)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

dataset = WithWithoutSpecs(csv_file='datasets/02/train.csv',
                           root_dir='datasets/02/images', transform=transform)
train_dataset, test_dataset = random_split(dataset, [4000, 500])
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True)

Выведем несколько изображений, чтобы проверить, правильно ли у нас сформировался DataLoader

In [ ]:
images, labels = next(iter(train_loader))
image_shower(images, labels, n=6)

Загрузим нашу модель

Заморозим веса

И переопределим последний слой на два класса

In [ ]:
model = torchvision.models.resnet18(pretrained=True)

for param in model.parameters():
    param.require = False

model.fc = nn.Sequential(nn.Linear(512, OUTPUT_SIZE) , nn.Sigmoid())
model.to(device);  # переносим нашу модель на GPU

Дообучим сеть на нашем датасете

In [ ]:
criterion = nn.CrossEntropyLoss()
accuracy = Accuracy().to(device)
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.1, alpha=0.9)

In [ ]:
run_train()

### Предскажем классы на валидационной выборке

In [ ]:
import os
import numpy as np
from skimage import io

import torch
import torch.nn as nn
import torch.functional as F
import torchvision
import torchvision.transforms as transforms


# путь к нашей модели, которую мы будем использовать для предсказания
PATH_TO_MODEL = 'model.pth'
# путь к изображениям, которые необходимо классифицировать
PATH_TO_IMAGES = 'images'


transform = transforms.Compose([transforms.ToPILImage(), 
                                transforms.Resize((64, 64)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])


# загружаем обученную модель
model = torch.load(PATH_TO_MODEL)
model.eval()

for img_name in os.listdir(PATH_TO_IMAGES):
    img_path = os.path.join(PATH_TO_IMAGES, img_name)
    image = transform(io.imread(img_path)).view(1, 3, 64, 64)
    output = model(image)
    _, prediction = torch.max(output, 1)
    
    with open('predictions.csv', 'a') as f:
        f.write(img_name + ';' + str(prediction.item()) + '\n')

### Посмотрим на результат

In [ ]:
import pandas as pd


df = pd.read_csv('predictions.csv', sep=';', header=None, names=['image', 'prediction'])
df.tail(15)